### Pulsed NMR

This notebook contains the code for the pulsed NMR system.

Load the required libraries.

In [ ]:
#r "nuget:ScottPlot"

using System.Numerics;
using Microsoft.DotNet.Interactive.Formatting;

#!import PulsedNMR.cs

Connect to the TCP server running on the Red Pitaya board.

In [ ]:
var c = new Client();
c.Connect("192.168.1.100");

Set frequencies, ADC sample rate and CIC decimation rate.

In [ ]:
c.SetFreqs(tx: 1e6, rx: 1e6);

c.SetRates(adc: 122.88, cic: 48);

Define pulse sequence.

In [ ]:
var d = 97;  // initial read delay
var u = 10;  // unblank delay
var a = 10;  // a pulse length
var b = 20;  // b pulse length
var ab = 90;  // ab delay
var bb = 180;  // bb delay

c.ClearEvents(readDelay: d);

// sync
c.AddEvent(1, sync: 1);
// unblank delay
c.AddEvent(u, gate: 1, level: 0, read: 1);
// a pulse
c.AddEvent(a, gate: 1, level: 100, read: 1);
// ab delay
c.AddEvent(ab - u, read: 1);
// unblank delay
c.AddEvent(u, gate: 1, level: 0, read: 1);

for (int i = 0; i < 5; i++)
{
  // b pulse
  c.AddEvent(b, gate: 1, level: 100, read: 1);
  // bu delay
  c.AddEvent(bb - u, read: 1);
  // unblank delay
  c.AddEvent(u, gate: 1, level: 0, read: 1);
}

Run one pulse sequence.

In [ ]:
var data = c.ReadData();

Plot signal.

In [ ]:
var complexData = data
  .Select((v, i) => new { value = v, index = i })
  .GroupBy(v => v.index / 2)
  .Select(g => new Complex(g.First().value, g.Last().value))
  .ToArray();

var in1 = complexData.Where((_, i) => i % 2 == 0).ToArray();

var magnitudes = in1.Select(v => v.Magnitude).ToArray();

var time = Enumerable.Range(0, in1.Length).Select(v => v * c.dt).ToArray();

Formatter.Register(typeof(ScottPlot.Plot), (p, w) =>
  w.Write(((ScottPlot.Plot)p).GetPngHtml(1200, 400)), HtmlFormatter.MimeType);

ScottPlot.Plot plt = new();
var scatter = plt.Add.Scatter(time, magnitudes);
scatter.LineWidth = 2;
scatter.MarkerSize = 0;
plt.XLabel("time, us");

plt

Disconnect from the TCP server.

In [ ]:
c.Disconnect();